In [1]:
import numpy as np
import pandas as pd
import gensim 
import logging

## Creating Corpus

### Extracting Text

go to https://en.wikipedia.org/wiki/Special:Export and export pages you want

In [2]:
cd ../libraries/wikiextractor

/Users/joshua/Development/business-analysis/libraries/wikiextractor


In [3]:
!python3 WikiExtractor.py -o ../../data/wiki/ --no-templates --processes 8 ../../data/screw_corpus.xml

INFO: Starting page extraction from ../../data/screw_corpus.xml.
INFO: Using 8 extract processes.
INFO: 6810986	ISO metric screw thread
INFO: 1588451	Screw thread
INFO: 27592945	List of screw drives
INFO: 20691673	Screw
INFO: Finished 8-process extraction of 4 articles in 0.1s (33.9 art/s)


In [4]:
cd ../../data/wiki/AA


/Users/joshua/Development/business-analysis/Data/wiki/AA


In [5]:
ls

wiki_00


In [6]:
data_file = 'wiki_00'

### Preprocessing Text

In [7]:
def read_input(input_file):
    logging.info("reading file {0}...this may take a while".format(input_file))
    f = open(input_file, 'r')
    for line in f:
        yield gensim.utils.simple_preprocess(line)
documents = list (read_input(data_file))

In [8]:
documents[3:5]

[['the',
  'iso',
  'metric',
  'screw',
  'threads',
  'are',
  'the',
  'worldwide',
  'most',
  'commonly',
  'used',
  'type',
  'of',
  'general',
  'purpose',
  'screw',
  'thread',
  'they',
  'were',
  'one',
  'of',
  'the',
  'first',
  'international',
  'standards',
  'agreed',
  'when',
  'the',
  'international',
  'organization',
  'for',
  'standardization',
  'iso',
  'was',
  'set',
  'up',
  'in'],
 []]

## Train the hidden layer for word vectors

In [9]:
model = gensim.models.Word2Vec(documents, size=48, window=15, min_count=2, workers=15, batch_words=5)
print(model)

Word2Vec(vocab=1406, size=48, alpha=0.025)


In [10]:
model.train(documents,total_examples=len(documents),epochs=10)

(111713, 173130)

In [17]:
example_word = 'drive'
model.wv.most_similar (positive=example_word)

[('head', 0.99969482421875),
 ('square', 0.9995798468589783),
 ('slotted', 0.9995594024658203),
 ('phillips', 0.9995234608650208),
 ('torx', 0.999517560005188),
 ('screwdriver', 0.9995139837265015),
 ('robertson', 0.9995015263557434),
 ('tool', 0.9994959235191345),
 ('drives', 0.9994714260101318),
 ('driver', 0.9994105100631714)]

## Load Part Number Descriptions

In [1]:
!pwd

/Users/joshua/Development/business-analysis/Tutorials


In [2]:
cd ../data

/Users/joshua/Development/business-analysis/Data


In [3]:
ls

BlackFriday.csv*		sales_data_sample.csv
customer_data.xlsx		sales_data_sample.xlsx
government_purchase_orders.csv	sales_data_sample_no_customer.xlsx
inventory_list.csv		sales_encryption.csv
part_usage_trailing_12.xlsx	screw_corpus.xml
purchases_by_vendor.xlsx


In [ ]:
inventory = pd.read_csv('inventory_list.csv')

In [7]:
inventory.head()

,Unnamed: 0,part_number,class,descrip,descrip2,cost,onhand,onorder,master_list,custno,revision,aql,syscom,physical,oldcost,owner
0,0,5f08e3cf2e05,NaN,MS20822-4,NaN,9.48626,2129,0,Y,17580.0,F,1,NaN,N,0.0,A
1,1,37062f157a8c,NaN,"1/4-28 x 10.313 Machine Screw, 1022 Steel",B/O(Dwg 18571);97022; HTS 7318.15.0000,0.19000,162000,0,Y,14700.0,19,1,NaN,N,0.0,A
2,2,9fafb0891d5f,NaN,Artemis Heat Sink,NaN,0.00000,0,0,NaN,NaN,NaN,0,NaN,N,0.0,C
3,3,4169ab8d76a6,NaN,"M8 X 1.25-6g X 40MM HHCS, BRASS",W/ BRITE DIP TO PRINT,0.21900,49100,0,NaN,NaN,NaN,0,NaN,N,0.0,A
4,4,e30c8087f64d,NaN,1/4-20 X 5/8 Unslotted Hex Hd Screw w/ L/W & F/W,18-8 Stainless Steel per F593C,0.00000,0,0,NaN,NaN,NaN,0,NaN,N,0.0,A


In [17]:
parts_list = inventory.replace(np.nan, ' ', regex=True)
parts_list.head()

,Unnamed: 0,part_number,class,descrip,descrip2,cost,onhand,onorder,master_list,custno,revision,aql,syscom,physical,oldcost,owner,description
0,0,5f08e3cf2e05,,MS20822-4,,9.48626,2129,0,Y,17580,F,1,,N,0.0,A,
1,1,37062f157a8c,,"1/4-28 x 10.313 Machine Screw, 1022 Steel",B/O(Dwg 18571);97022; HTS 7318.15.0000,0.19000,162000,0,Y,14700,19,1,,N,0.0,A,"1/4-28 x 10.313 Machine Screw, 1022 Steel B/O(..."
2,2,9fafb0891d5f,,Artemis Heat Sink,,0.00000,0,0,,,,0,,N,0.0,C,
3,3,4169ab8d76a6,,"M8 X 1.25-6g X 40MM HHCS, BRASS",W/ BRITE DIP TO PRINT,0.21900,49100,0,,,,0,,N,0.0,A,"M8 X 1.25-6g X 40MM HHCS, BRASS W/ BRITE DIP T..."
4,4,e30c8087f64d,,1/4-20 X 5/8 Unslotted Hex Hd Screw w/ L/W & F/W,18-8 Stainless Steel per F593C,0.00000,0,0,,,,0,,N,0.0,A,1/4-20 X 5/8 Unslotted Hex Hd Screw w/ L/W & F...


In [20]:
parts_list['description'] = parts_list['descrip'] + ' ' + parts_list['descrip2']

In [21]:
parts_list['description'].head()

0                                          MS20822-4  
1    1/4-28 x 10.313 Machine Screw, 1022 Steel B/O(...
2                                  Artemis Heat Sink  
3    M8 X 1.25-6g X 40MM HHCS, BRASS W/ BRITE DIP T...
4    1/4-20 X 5/8 Unslotted Hex Hd Screw w/ L/W & F...
Name: description, dtype: object